In [9]:
import pandas as pd
import json
import gensim
import glob
from gensim.models.doc2vec import Doc2Vec
import jieba


def news_push(data, most_views_news1, most_views_news2, most_views_news3, most_views_news4):
    df_all = pd.DataFrame(columns=["title_id", "origin_id", "title", "url", "content"])
    df = pd.DataFrame.from_dict(data, orient="columns")
    df_all = pd.concat([df_all, df], axis=0, ignore_index=True)

    df = df_all
    df = df.drop(["url"], axis=1)
    df = df.drop(["origin_id"], axis=1)
    df = df.drop(["title"], axis=1)

    def split_news(news):
        return " ".join(jieba.cut(news))

    def process_news(df):
        df['content'] = df['content'].apply(split_news)
        return df

    df = process_news(df)

    TaggededDocument = gensim.models.doc2vec.TaggedDocument

    news_list = []
    for i in range(0, len(df)):
        news = list(df.iloc[i])
        news_list.append(news)

    x_train = []
    for content, title_id in news_list:
        word_list = content.split(' ')
        l = len(word_list)
        word_list[l - 1] = word_list[l - 1].strip()
        document = TaggededDocument(word_list, tags=[title_id])
        x_train.append(document)
    c = x_train

    model = Doc2Vec(x_train, min_count=1, window=3, vector_size=100, negative=5, workers=4)
    model.train(x_train, total_examples=model.corpus_count, epochs=10)
    model_dm = model

    # 第一個推播
    strl = df["title_id"] == most_views_news1
    news_index = int(str(df[strl]["content"]).split(" ")[0])
    test_text = df[strl]["content"][news_index].split(' ')
    # 得到向量array命名為inferred_vector
    inferred_vector = model_dm.infer_vector(doc_words=test_text, alpha=0.025, steps=500)

    sims = model_dm.docvecs.most_similar([inferred_vector], topn=3)

    title_id_list1 = []
    for title_id, sim in sims:
        title_id_list1.append(title_id)

    # 第二個推播
    strl = df["title_id"] == most_views_news2
    news_index = int(str(df[strl]["content"]).split(" ")[0])
    test_text = df[strl]["content"][news_index].split(' ')
    # 得到向量array命名為inferred_vector
    inferred_vector = model_dm.infer_vector(doc_words=test_text, alpha=0.025, steps=500)

    sims = model_dm.docvecs.most_similar([inferred_vector], topn=3)

    title_id_list2 = []
    for title_id, sim in sims:
        title_id_list2.append(title_id)

    # 第三個推播
    strl = df["title_id"] == most_views_news3
    news_index = int(str(df[strl]["content"]).split(" ")[0])
    test_text = df[strl]["content"][news_index].split(' ')
    # 得到向量array命名為inferred_vector
    inferred_vector = model_dm.infer_vector(doc_words=test_text, alpha=0.025, steps=500)

    sims = model_dm.docvecs.most_similar([inferred_vector], topn=3)

    title_id_list3 = []
    for title_id, sim in sims:
        title_id_list3.append(title_id)

    # 第四個推播
    strl = df["title_id"] == most_views_news4
    news_index = int(str(df[strl]["content"]).split(" ")[0])
    test_text = df[strl]["content"][news_index].split(' ')
    # 得到向量array命名為inferred_vector
    inferred_vector = model_dm.infer_vector(doc_words=test_text, alpha=0.025, steps=500)

    sims = model_dm.docvecs.most_similar([inferred_vector], topn=3)

    title_id_list4 = []
    for title_id, sim in sims:
        title_id_list4.append(title_id)

    return (title_id_list1, title_id_list2, title_id_list3, title_id_list4)

In [10]:
import requests
import json

def url_change_titleid(url):
    ip_location='chatbot_api'
    playload = {"url" : url}
    Endpoint='http://%s:5000/get_title_id/' % (ip_location)

    # header要特別註明是json格式
    Header={'Content-Type':'application/json'}

    # 傳送post對API server新增資料 
    Response=requests.post(Endpoint,headers=Header,data=json.dumps(playload))

    #印出Response的資料訊息

    return Response.json()['title_id']

In [ ]:
import requests
import json

def if_have_img(title_id):
    ip_location='chatbot_api'
    playload = {"url" : url}
    Endpoint='http://%s:5000/get_title_id/' % (ip_location)

    # header要特別註明是json格式
    Header={'Content-Type':'application/json'}

    # 傳送post對API server新增資料 
    Response=requests.post(Endpoint,headers=Header,data=json.dumps(playload))

    #印出Response的資料訊息

    return Response.json()['title_id']

In [1]:
import requests
import datetime
from bs4 import BeautifulSoup

if_end = True
store_datetime = datetime.datetime.now().strftime("%Y/%m/%d")
my_json = []
for page in range(1, 12):
    if if_end == False:
        break
    href = "https://tw.news.appledaily.com/politics/realtime/"+ str(page)
    res = requests.get(href)
    html = BeautifulSoup(res.text)
    all_news_1 = html.find_all("ul", class_="rtddd slvl")
    for all_news in all_news_1:
        if if_end == False:
            break
        news = all_news.find_all("a")
        for n in news:
            my_news = {}
            news_url ="https://tw.news.appledaily.com/" + str(n["href"])
            news_per = requests.get(news_url)
            bs = BeautifulSoup(news_per.text)

            if bs.find("div", class_="ndArticle_view") == None:
                views = 0
            else:
                views = int(bs.find("div", class_="ndArticle_view").text)
            date_time = bs.find("div", class_="ndArticle_creat").text.replace("出版時間：", "").split(" ")[0]
            if not date_time == store_datetime:
                if_end = False
                break
            my_news= {"url": news_url, "views": views}
            my_json.append(my_news)
            print(my_news)

sorted_json = sorted(my_json ,key = lambda my_json:my_json['views'], reverse = True)



{'url': 'https://tw.news.appledaily.com//politics/realtime/20190107/1496454/', 'views': 44}
{'url': 'https://tw.news.appledaily.com//politics/realtime/20190107/1496333/', 'views': 158}
{'url': 'https://tw.news.appledaily.com//politics/realtime/20190107/1496436/', 'views': 234}
{'url': 'https://tw.news.appledaily.com//politics/realtime/20190107/1496385/', 'views': 12154}
{'url': 'https://tw.news.appledaily.com//politics/realtime/20190107/1496284/', 'views': 4143}
{'url': 'https://tw.news.appledaily.com//politics/realtime/20190107/1496412/', 'views': 1696}
{'url': 'https://tw.news.appledaily.com//politics/realtime/20190107/1496290/', 'views': 2230}
{'url': 'https://tw.news.appledaily.com//politics/realtime/20190107/1496418/', 'views': 4453}
{'url': 'https://tw.news.appledaily.com//politics/realtime/20190107/1496411/', 'views': 1201}
{'url': 'https://tw.news.appledaily.com//politics/realtime/20190107/1496405/', 'views': 7432}
{'url': 'https://tw.news.appledaily.com//politics/realtime/2019

In [12]:
url_change_titleid(sorted_json[0]['url'])


4864

In [14]:
url_change_titleid(sorted_json[1]['url'])

4897

In [15]:
url_change_titleid(sorted_json[2]['url'])

4889

In [16]:
url_change_titleid(sorted_json[3]['url'])

4891

In [8]:
import requests
import json

Endpoint='http://chatbot_api:5000/today_news'

todaynews = requests.get(Endpoint)
data = todaynews.json()

In [23]:
p1, p2, p3, p4 = news_push(data, url_change_titleid(sorted_json[0]['url']), url_change_titleid(sorted_json[1]['url']), url_change_titleid(sorted_json[2]['url']), url_change_titleid(sorted_json[3]['url']))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [25]:
all_push_news= news_push(data, url_change_titleid(sorted_json[0]['url']), url_change_titleid(sorted_json[1]['url']), url_change_titleid(sorted_json[2]['url']), url_change_titleid(sorted_json[3]['url']))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
def if_exit_img(my_list):
    my_list[0]

In [29]:
print(all_push_news)
push_news = []
for a in all_push_news:
    if len(push_news) < 3:
        if len(a) == 3:
            push_news.append(a)


([5069, 5107, 4970], [4897, 4852, 4765], [4788, 4889, 4995], [4891, 4786, 4997])
